In [54]:
import importlib

import thermo_flux

import numpy as np
import pandas as pd

from thermo_flux.utils.annotation import get_suitable_ids
from equilibrator_api import ComponentContribution, Q_
from thermo_flux.core.model import ThermoModel
importlib.reload(thermo_flux.core.model)


<module 'thermo_flux.core.model' from 'C:\\Users\\tedns\\Google Drive\\Thermodynamic plants\\thermo_flux\\thermo_flux\\core\\model.py'>

In [55]:
importlib.reload(thermo_flux.utils.annotation)
from thermo_flux.utils.annotation import get_suitable_ids


In [56]:
importlib.reload(thermo_flux)
importlib.reload(thermo_flux.core.model)
importlib.reload(thermo_flux.core.metabolite)

from thermo_flux.core.model import ThermoModel
from thermo_flux.core.metabolite import ThermoMetabolite



In [57]:
#cc = ComponentContribution()

In [58]:
import cobra
model = cobra.io.read_sbml_model('ecoli_test.xml')
tmodel = ThermoModel(model)


ERROR:cobra.io.sbml:No objective coefficients in model. Unclear what should be optimized


In [59]:
for met in tmodel.metabolites:
    cpd, annotation, formula, inchi = get_suitable_ids(met,cc)
    met.annotation.update(annotation)
    met.compound = cpd

In [60]:
tmodel.metabolites[0].accession = None
tmodel.metabolites[0].accession

'metanetx.chemical:MNXM42'

In [61]:
#tmodel.metabolites.charge_c.accession = 'test'
tmodel.metabolites.charge_e.accession

'charge'

In [62]:
for met in tmodel.metabolites:
    if met.compound is None:
        print(met.id)

proteins_c
lipids_c
polysacch_c
dna_c
rna_c
storage_c
charge_e
charge_c
charge_m


In [63]:
#tmodel.metabolites.charge_c.dfG_SE = 100
print(tmodel.metabolites.charge_c.dfG_SE)
print(tmodel.metabolites.charge_e.dfG_SE)

#tmodel.metabolites.charge_e.dfG_SE = 100
print(tmodel.metabolites.charge_c.dfG_SE)
print(tmodel.metabolites.charge_e.dfG_SE)

None
None
None
None


In [64]:
tmodel.metabolites.focytc_c.redox = True
print(tmodel.metabolites.focytc_c.redox )
tmodel.metabolites.focytc_c.dfG0prime = 12.25 

True


In [79]:
tmodel.metabolites.biomass_c.biomass = True
tmodel.metabolites.biomass_e.biomass = True
tmodel.metabolites.biomass_c.dfG0prime = 1000
tmodel.metabolites.biomass_c.dfG0prime

1000

In [66]:
from thermo_flux.drg_tools import drg_tools
importlib.reload(drg_tools)


<module 'thermo_flux.drg_tools.drg_tools' from 'C:\\Users\\tedns\\Google Drive\\Thermodynamic plants\\thermo_flux\\thermo_flux\\drg_tools\\drg_tools.py'>

In [67]:
drg_tools.get_compound(tmodel.metabolites[0].compound, cc)

Compound(id=44, inchi_key=CKLJMWTZIZZHCS-REOHCLBHSA-M)

In [68]:
print(type(tmodel.metabolites[0]))
print(type(ThermoMetabolite))


<class 'thermo_flux.core.metabolite.ThermoMetabolite'>
<class 'type'>


In [69]:
from equilibrator_api import Reaction

In [70]:
#tmodel.metabolites.charge_c.dfG_SE =100

In [80]:
from equilibrator_api import Reaction

def calc_dfG0(tmodel, cc):
    default_dfg0_std = Q_(10000,"kJ/mol")

    known_met_idx = []
    unknown_met_idx = []
    unknown_mets = []

    dfG0_mean = np.zeros((len(tmodel.metabolites), 1))
    dfG0_cov_sqrt = np.zeros((len(tmodel.metabolites), len(tmodel.metabolites)))


    known_met_list = []

    for i, met in enumerate(tmodel.metabolites):
        if met.compound is None:
            unknown_met_idx.append(i)
            unknown_mets.append(met)
        else:
            known_met_idx.append(i)
            known_met_list.append(Reaction({met.compound:1}))

    dfG0_mean_1, dfG0_cov_sqrt_1 = cc.standard_dg_multi(known_met_list,"fullrank")

    dfG0_mean[known_met_idx,0] = dfG0_mean_1.m
    dfG0_cov_sqrt[known_met_idx, 0:dfG0_cov_sqrt_1.shape[1]] = dfG0_cov_sqrt_1.m


    #set unknown metabolites to have a reasonable formation energy
    #is the mean of other formation energies reasonable? 
    #would 0 be a better estimate?
    #is this very case specific i.e generic proteins etc. are note normal metabolites 
    default_dfg0_mean = np.mean(dfG0_mean[known_met_idx]) #this value is not needed becuase of the fitting later?
    dfG0_mean[unknown_met_idx] = 0 #set unknown metabolites to have formation energy of 0 unless specified in tmet
    
    
    
    #for emtabolites which have an identifier in the equilibrator cache but we want to ignore it and change the uncertainty
    #only for redox carrier and biomass can we set our own formation energy 
    #aslo for photons 
    #can only do this for metabolites that are not correlated with any others (apart from themselves)
    
    special_mets_idx = []    
    for i, met in enumerate(tmodel.metabolites):
        if any([met.redox, met.biomass]):
            if len(np.nonzero(dfG0_cov_sqrt[i])[0]) <= 1:    #only metabolites with a single entry in dfG0_cov_sqrt
                if met.dfG0prime is None:
                    dfG_mean = 0
                else:
                    dfG_mean = met.dfG0prime #this shold be a feature of a reaction not a metabolite - stoichometry need accounting for?
                special_mets_idx.append(i)
                dfG0_mean[i] = dfG_mean
                if met.dfG_SE is None:
                    dfG_SE = 0
                else:
                    dfG_SE = met.dfG_SE
                unknown_basis_idx = (np.abs(dfG0_cov_sqrt[i])).argmax()
                dfG0_cov_sqrt[i,unknown_basis_idx] = dfG_SE
                
                
    unknown_basis_idx = [i for i in range(dfG0_cov_sqrt.shape[1]) if np.any(np.abs(dfG0_cov_sqrt[:, i]) >= default_dfg0_std.m)]
    
    #enforce correlation of multiple instances of unknown metabolites in different compartments 
    dfG_basis_size = dfG0_cov_sqrt_1.shape[1]
    unknown_met_basis_id = {}
    

    for met_idx in unknown_met_idx:
        if met_idx not in special_mets_idx:
            met = tmodel.metabolites[met_idx]
            if met.accession in unknown_met_basis_id:     #if the metabolite already has column in the unknown basis
                if met.dfG_SE is None:
                    dfG_SE = default_dfg0_std.m
                else:
                    dfG_SE = met.dfG_SE

                dfG0_cov_sqrt[met_idx, unknown_met_basis_id[met.accession]] = dfG_SE

            else:#otherwise add a new basis for the unknown metabolite 

                unknown_met_basis_id[met.accession] = dfG_basis_size
                if met.dfG_SE is None:
                    dfG_SE = default_dfg0_std.m
                else:
                    dfG_SE = met.dfG_SE


                dfG0_cov_sqrt[met_idx, dfG_basis_size] = dfG_SE
                dfG_basis_size += 1


            
    #return unknown basis for fitting later
    unknown_basis_idx += range(dfG0_cov_sqrt_1.shape[1], dfG_basis_size)
    unknowns_basis = dfG0_cov_sqrt[:, unknown_basis_idx]


    return dfG0_mean, dfG0_cov_sqrt, unknowns_basis

In [81]:
dfG0_mean, dfG0_cov_sqrt, unknowns_basis = calc_dfG0(tmodel, cc)

In [82]:
import numpy.linalg as la

S = cobra.util.array.create_stoichiometric_matrix(tmodel)

X = S.T @ unknowns_basis
y = -S.T @ dfG0_mean
beta = la.lstsq(X, y, rcond=None)[0]
dfG0_mean_2 = dfG0_mean + unknowns_basis @ beta


In [83]:
(unknowns_basis @ beta)[198]

array([381.78019665])

In [84]:
model.metabolites.index(model.metabolites.biomass_c)

227

In [85]:
df = pd.DataFrame(dfG0_cov_sqrt, index=[met.id for met in model.metabolites])
df.loc[['ficytc_c','focytc_c']]

,0,1,2,3,4,5,6,7,8,9,...,219,220,221,222,223,224,225,226,227,228
ficytc_c,0.0,0.0,-100000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
focytc_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
df[227].sort_values(key=abs,ascending=False)

asp-L_c      0.0
hcit_m       0.0
L2aadp_c     0.0
b124tc_m     0.0
hicit_m      0.0
            ... 
hcys-L_c     0.0
nad_c        0.0
nadh_c       0.0
for_c        0.0
biomass_e    0.0
Name: 227, Length: 229, dtype: float64

In [88]:
dfG0_mean[223]

array([12.25])

In [ ]:
dfG0_cov_sqrt[known_met_idx, 0:dfG0_cov_sqrt_1.shape[0]]


In [ ]:
dfG0_cov_sqrt.shape

In [ ]:
dfG0_cov_sqrt.shape[1]

In [ ]:
dfG0_cov_sqrt.m[:,1].argmin()

In [ ]:
model.metabolites[123]

In [ ]:
unreliable_basis_ids

In [ ]:
tmodel.metabolites[1].annotation

In [ ]:
tmodel.metabolites.polysacch_c.compound.get_accession()

In [ ]:
dfG_mean, dfG_cov = cc.standard_dg_multi(met_list,"fullrank")

In [ ]:
len(dfG_mean)

In [ ]:
dfG_cov.shape

In [ ]:
unreliable_basis_ids = [i
                for i in range(dfG_cov.shape[1])
                if np.any(np.abs(dfG_cov[:, i]) >= Q_(10000,"kJ/mol"))
            ]


In [ ]:
unreliable_basis_ids

In [ ]:
dfG_cov.m[:,11]

In [ ]:
default_dfg0_mean = np.mean(dfG_mean)


In [ ]:
default_dfg0_mean

In [ ]:
dfG_mean

In [ ]:
import pandas as pd

In [ ]:
tmodel.metabolites.charge_m.compound.get_common_name()

In [ ]:
tmodel.metabolites.charge_c.compound.get_common_name()

In [ ]:
df = pd.DataFrame(dfG_cov.m, index=[met.id for met in model.metabolites])
df.loc['']

In [ ]:
ThermoMetabolite

#redox_couple = True/False
#Redox_partner = ThermoMetabolite (in model)
#midpont potential = 
#midpoint potential SE = 

#thsi could allow thermodynamically incocnistent cycles?


#manually update the midpoint potential of a redox carrier in volts
#calcualtions then consider this... 
#need to make sure if a value is set in the midpoint potnetial field the metabolite has no compound 
#in the thermo basis it needs dfG0' set by data 
#can midpoint potnetials be transformed to different conditions?
#give a standard midpoint potential 
    #number of electrons and ptotons known
    #calcualte transformed 
    #or give trsnaformed and calcualte back to normal?
    
    
#warning potential thermodynamic inconsistency ensumre redox compounds are not synthesised in model 

In [ ]:
def calc_dfG0(mets, cc):
    
    #make list of compound objects 
    #identify completely unknown metabolites 
    #identify specified redox carriers #hack until equilibrator assest can add new redox compounds 
    
    
    #calcualte dfG
    
    #populate completely unkown values with more reasonable estimates
    
    #populate redox carriers with data 
    
    #adjust covariance to account for redox carriers 
    
    #enforce correlation for unknown metabolites 
        #could check that metabolites with the same name or same id[:-2] have the same compound 
        #and metabolites with same name or id[:-2] are correlated in dfG space 

In [ ]:
def calc_dfG0_prime(mets, cc):

In [ ]:
#fit unknown dfG0